In [5]:
import numpy as np
import matplotlib.pyplot as plt
import scipy, scipy.signal

# from scipy.signal import butter, lfilter
# from santolik_methods import santolik_Q, appleton_hartree, ne_ps, gen_rotation_matrix
from santolik_methods import gen_td_whistler, santolik_Q

In [6]:
# import constants
c = 2.998e8 # m/s 
D2R = np.pi/180.
R2D = 180./np.pi
Hz2Rad = 2.*np.pi
Rad2Hz = 1./Hz2Rad
Q_EL = 1.602e-19
M_EL = 9.1e-31
eo   = 8.854e-12
B0   = 30e-6
mu   = 4.0*np.pi*10e-7

In [8]:
def plot_td(ex, ey, ez, bx, by, bz, fs):
    tvec = np.arange(0,len(ex),1)/fs
    fig, ax = plt.subplots(3,2, sharex=True, sharey='col')
    ax[0,0].plot(tvec, ex)
    ax[1,0].plot(tvec, ey)
    ax[2,0].plot(tvec, ez)
    ax[0,1].plot(tvec, bx)
    ax[1,1].plot(tvec, by)
    ax[2,1].plot(tvec, bz)
    fig.autofmt_xdate()
    fig.tight_layout()
    ax[0,0].set_title('E')
    ax[0,1].set_title('B')
def plot_spec(ex, ey, ez, bx, by, bz, fs, E_clims = [-12, -4], B_clims = [-20, -15]):
    # --------------- Latex Plot Beautification --------------------------
    fig_width = 8
    fig_height = 6
    fig_size =  [fig_width+1,fig_height+1]
    params = {'backend': 'ps',
              'axes.labelsize': 12,
              'font.size': 12,
              'legend.fontsize': 10,
              'xtick.labelsize': 12,
              'ytick.labelsize': 12,
              'text.usetex': False,
              'figure.figsize': fig_size}
    plt.rcParams.update(params)
    # --------------- Latex Plot Beautification --------------------------
    fig, ax = plt.subplots(3,2,sharex=True,sharey=True)

    # evec = [ex,ey,ez]
    # bvec = [bx,by,bz]
    window = 'hanning'
    nfft = 128
    overlap = 0.5
    c = 2.998e8
    shading = 'flat'
#         shading = 'gouraud'
    # Get spectra
    ff,tt, FBx = scipy.signal.spectrogram(bx, fs=fs, window=window, nperseg=nfft, noverlap=nfft*overlap,mode='complex')
    ff,tt, FBy = scipy.signal.spectrogram(by, fs=fs, window=window, nperseg=nfft, noverlap=nfft*overlap,mode='complex')
    ff,tt, FBz = scipy.signal.spectrogram(bz, fs=fs, window=window, nperseg=nfft, noverlap=nfft*overlap,mode='complex')

    ff,tt, FEx = scipy.signal.spectrogram(1e-3*ex, fs=fs, window=window, nperseg=nfft, noverlap=nfft*overlap,mode='complex')
    ff,tt, FEy = scipy.signal.spectrogram(1e-3*ey, fs=fs, window=window, nperseg=nfft, noverlap=nfft*overlap,mode='complex')
    ff,tt, FEz = scipy.signal.spectrogram(1e-3*ez, fs=fs, window=window, nperseg=nfft, noverlap=nfft*overlap,mode='complex')

    S_mag = np.sqrt(np.real(FEx*np.conj(FEx)))
#     print(np.min(S_mag), np.max(S_mag))
    p = ax[0,0].pcolormesh(tt,ff, np.log10(S_mag), cmap = plt.cm.jet,vmin=E_clims[0],vmax=E_clims[1],shading=shading)
    S_mag = np.sqrt(np.real(FEy*np.conj(FEy)))
#     print(np.min(S_mag), np.max(S_mag))
    p = ax[1,0].pcolormesh(tt,ff, np.log10(S_mag), cmap = plt.cm.jet,vmin=E_clims[0],vmax=E_clims[1],shading=shading)
    # S_mag = np.real(newEz*np.conj(newEz))
    S_mag = np.sqrt(np.real(FEz*np.conj(FEz)))
#     print(np.min(S_mag), np.max(S_mag))
    p = ax[2,0].pcolormesh(tt,ff, np.log10(S_mag), cmap = plt.cm.jet,vmin=E_clims[0],vmax=E_clims[1],shading=shading)

    S_mag = np.sqrt(np.real(FBx*np.conj(FBx)))
#     print(np.min(S_mag), np.max(S_mag))
    p = ax[0,1].pcolormesh(tt,ff, np.log10(S_mag), cmap = plt.cm.jet,vmin=B_clims[0],vmax=B_clims[1],shading=shading)
    S_mag = np.sqrt(np.real(FBy*np.conj(FBy)))
#     print(np.min(S_mag), np.max(S_mag))
    p = ax[1,1].pcolormesh(tt,ff, np.log10(S_mag), cmap = plt.cm.jet,vmin=B_clims[0],vmax=B_clims[1],shading=shading)
    S_mag = np.sqrt(np.real(FBz*np.conj(FBz)))
#     print(np.min(S_mag), np.max(S_mag))
    p = ax[2,1].pcolormesh(tt,ff, np.log10(S_mag), cmap = plt.cm.jet,vmin=B_clims[0],vmax=B_clims[1],shading='gouraud')

    ax[0,0].set_ylim([0,fs/2])
    ax[0,0].set_title('E')
    ax[0,1].set_title('B')
    ax[0,0].set_ylabel('X\nfrequency [hz]')
    ax[1,0].set_ylabel('Y\nfrequency [hz]')
    ax[2,0].set_ylabel('Z\nfrequency [hz]')
    ax[2,0].set_xlabel('Time [sec]')

    return fig


In [9]:
tmax = 0.5 # 0.2 # seconds
fs = 131072 # 2^17 (next power of 2 above 100kHz)

Lshell = 1.3
D = 5#2
theta = 40*D2R
phi = -45*D2R
pol_angle = 0
E_mag = 1

whistler_highcut = 40000
ex, ey, ez, bx, by, bz = gen_td_whistler(Lshell, D, theta, phi, pol_angle, tmax, fs, highcut=whistler_highcut,
                                         E_mag = 10e-3, e_noise_mag=1e-5, b_noise_mag = 1e-15)
print(ex)

In [14]:

# plot_td(tvec, ex, ey, ez, bx, by, bz)
# plot_spec(ex, ey, ez, bx, by, bz, fs)

fig = plot_spec(ex, ey, ez, bx, by, bz, fs)
fig.suptitle(r'Synthesized Whistler: L = {0:g}, D = {1:d}, $\theta$={2:g}, $\phi$={3:g}'.format(Lshell, D, R2D*theta, R2D*phi))
fig.savefig('whistler_spec.png')
fig = plot_td(ex, ey, ez, bz, by, bz, fs)
plt.show()

AttributeError: 'NoneType' object has no attribute 'show'